In [1]:
import itertools as it

import mbuild as mb
from mbuild.compound import Compound
from mbuild.coordinate_transform import force_overlap
from mbuild.utils.validation import assert_port_exists
from mbuild import clone
from copy import deepcopy
from visualize import visualize

In [2]:
__all__ = ['Pores']

In [42]:
class Pores(mb.Compound):
    """A general slit pore recipe
    
    Parameters
    ----------
    x_pop: int
        population in x-direction
    y_pop: int
        population in y-direction
    z_pop: int
        population in z-direction
    pore_size: int
        size of pore (nm)
    solvents: array
        solvents that fill the system
    
    Attributes
    ----------
    
    Notes: Match graphene y-dimension with box x-dimension
    """
    def __init__(self, replicate, dimension, solvent):
        super(Pores,self).__init__()
        self.replicate = replicate
        self.dimension = dimension
        if all(x <= 0 for x in dimension):
            raise ValueError('Dimension of box must be greater than 0')
        self.name = 'C'
        carbon_locations = [[0,0,0], [2/3,1/3,0]]
        basis = {self.name: carbon_locations}
        graphene_lattice = mb.Lattice(lattice_spacing=[.2456,.2456,.335], angles=[90,90,120], lattice_points=basis)
        carbon = mb.Compound(name=self.name)
        graphene = graphene_lattice.populate(compound_dict={self.name: carbon},
                                         x=replicate[0],y=replicate[1],z=replicate[2])
        print(graphene.periodicity)
        for particle in graphene.particles():
            if particle.xyz[0][0] < 0:
                particle.xyz[0][0] += graphene.periodicity[0]
        graphene.periodicity *= 13/15
        
        graphene.periodicity += self.dimension 
        print(graphene.periodicity)
        #graphene.periodicity[0] = self.dimension[2]
        #position_mult = self.replicate[2] - 2
        
        bottom_sheet = mb.clone(graphene)
        #bottom_sheet.translate([0, -(self.dimension[1])-(.335*4), 0]) 
        bottom_sheet.spin(1.5708,[1,0,0])
        top_sheet = mb.clone(graphene)
        box = mb.Box([21,7,7])
        #top_sheet.translate([0, (self.dimension[1]/2), 0]) 
        #top_sheet.translate([2, graphene.periodicity[1]/2-(.335*4), graphene.periodicity[2]/2]) 
        top_sheet.spin(1.5708,[1,0,0])
        system = mb.Compound()
        system.from_parmed(structure=bottom_sheet.to_parmed(box=deepcopy(box))+top_sheet.to_parmed(box=deepcopy(box)))
        #save system here
        self.solvent = solvent
        fill = mb.load(self.solvent)
        print(self.dimension)
        
        system = mb.solvate(system, fill, 10000, box=box, overlap=0.2)
        #system = mb.solvate(system, fill, 10000, box=mb.Box(graphene.periodicity), overlap=0.2)
        # Look at mb.Box(self.dimension)
        
        #system.periodicity = self.dimension
        self.add(system)

In [43]:
test=Pores(replicate=[40,31,3], dimension=[21,7,7], solvent='/Users/raymatsumoto/science/il_solvent_local/file_gen/mol2/tip3p.mol2')
test.save('system_test.gro',overwrite=overwrite=True)

/Users/raymatsumoto/mbuild/mbuild/lattice.py:527: UserWarning: Periodicity of non-rectangular lattices are not valid with default boxes. Only rectangular lattices are valid at this time.
  warn('Periodicity of non-rectangular lattices are not valid with '


[ 9.824   7.6136  1.005 ]
[ 29.51413333  13.59845333   7.871     ]


/Users/raymatsumoto/anaconda2/envs/hackathon/lib/python3.5/site-packages/pandas/io/parsers.py:1961: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/Users/raymatsumoto/anaconda2/envs/hackathon/lib/python3.5/site-packages/pandas/io/parsers.py:1963: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


[21, 7, 7]
